#### Original version of findCenter()
This function evaluates the circumcenter of the `P` points.If the points lies on a `d-1` circumball then the function is not able to perform the evaluation and therefore returns a `NaN` array.


In [ ]:
function findCenter(P::Lar.Points)::Array{Float64,1}
	dim, n = size(P)
	@assert n > 0		"findCenter: at least one points is needed."
	@assert dim >= n-1	"findCenter: Too much points"
	@assert dim < 4		"findCenter: Function not yet Programmed."

	if n == 1
		center = P[:, 1]

	elseif n == 2
		#for each dimension
		center = (P[:, 1] + P[:, 2]) / 2

	elseif n == 3
		#https://www.ics.uci.edu/~eppstein/junkyard/circumcenter.html
		if dim == 2
			denom = 2 * Lar.det([ P[:, 2] - P[:, 1]  P[:, 3] - P[:, 1] ])
			deter = (P[:, 2] - P[:, 1]) * Lar.norm(P[:, 3] - P[:, 1])^2 -
					(P[:, 3] - P[:, 1]) * Lar.norm(P[:, 2] - P[:, 1])^2
			numer = [- deter[2], deter[1]]
			center = P[:, 1] + numer / denom

		elseif dim == 3
			#circumcenter of a triangle in R^3
			numer = Lar.norm(P[:, 3] - P[:, 1])^2 * Lar.cross(
						Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]),
						P[:, 2] - P[:, 1]
					) +
					Lar.norm(P[:, 2] - P[:, 1])^2 * Lar.cross(
				  		P[:, 3] - P[:, 1],
						Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]
					)
			)
			denom = 2 * Lar.norm(
				Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1])
			)^2
			center = P[:, 1] + numer / denom
		end

	elseif n == 4 #&& dim = 3
		# https://people.sc.fsu.edu/~jburkardt/presentations
		#	/cg_lab_tetrahedrons.pdf
		# page 6 (matrix are transposed)
		α = Lar.det([P; ones(1, 4)])
		sq = sum(abs2, P, dims = 1)
		Dx = Lar.det([sq; P[2:2,:]; P[3:3,:]; ones(1, 4)])
		Dy = Lar.det([P[1:1,:]; sq; P[3:3,:]; ones(1, 4)])
		Dz = Lar.det([P[1:1,:]; P[2:2,:]; sq; ones(1, 4)])
		center = [Dx; Dy; Dz]/2α
	end

	return center
#	AlphaStructures.foundCenter([P[:,i] for i = 1 : size(P, 2)])[:,:]
end


#### Modified version
For parallelize this function, we subdivided it to four functions, as shown below:

In [ ]:
function firstMember(P::Lar.Points)::Array{Float64,1}
	n1=  Lar.norm(P[:, 3] - P[:, 1])^2 * Lar.cross(
				Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]),
				P[:, 2] - P[:, 1]
			)
	return n1
end

In [ ]:
function secondMember(P::Lar.Points)::Array{Float64,1}
	 n2=  Lar.norm(P[:, 2] - P[:, 1])^2 * Lar.cross(
	   P[:, 3] - P[:, 1],
	   Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]
		))
	return n2
end

In [ ]:
function denominatore(P::Lar.Points)::Float64
	d=	2 * ( Lar.norm(
			Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1])))^2
	return d
end

In [ ]:
function findCenter(P::Lar.Points)::Array{Float64,1}
	dim, n = size(P)
	@assert n > 0		"findCenter: at least one points is needed."
	@assert dim >= n-1	"findCenter: Too much points"
	@assert dim < 4		"findCenter: Function not yet Programmed."

	if n == 1
		center =  P[:, 1]

	elseif n == 2
		#for each dimension
		center =  (P[:, 1] + P[:, 2]) / 2

	elseif n == 3
		#https://www.ics.uci.edu/~eppstein/junkyard/circumcenter.html
		if dim == 2
			denom = 2 * Lar.det([ P[:, 2] - P[:, 1]  P[:, 3] - P[:, 1] ])
			deter = (P[:, 2] - P[:, 1]) * Lar.norm(P[:, 3] - P[:, 1])^2 -
					(P[:, 3] - P[:, 1]) * Lar.norm(P[:, 2] - P[:, 1])^2
			numer = [- deter[2], deter[1]]
			center = P[:, 1] + numer / denom

		elseif dim == 3
			#circumcenter of a triangle in R^3

			n1= @spawn firstMember(P)
			n2= @spawn secondMember(P)

			n1=fetch(n1)
			n2=fetch(n2)
			numer = n1+n2

			denom=@spawn denominatore(P)
			denom=fetch(denom)
			center = P[:, 1] + numer / denom
		end

	elseif n == 4 #&& dim = 3
		# https://people.sc.fsu.edu/~jburkardt/presentations
		#	/cg_lab_tetrahedrons.pdf
		# page 6 (matrix are transposed)
		α =Threads.@spawn Lar.det([P; ones(1, 4)])
		α=fetch(α)
		sq = sum(abs2, P, dims = 1)
		Dx = Threads.@spawn Lar.det([sq; P[2:2,:]; P[3:3,:]; ones(1, 4)])
		Dx= fetch(Dx)
		Dy =Threads.@spawn Lar.det([P[1:1,:]; sq; P[3:3,:]; ones(1, 4)])
		Dy= fetch(Dy)
		Dz =Threads.@spawn Lar.det([P[1:1,:]; P[2:2,:]; sq; ones(1, 4)])
		Dz=fetch(Dz)
		center = [Dx; Dy; Dz]/2α
	end

	return center
end

#### Original Version of findClosestPoint
This function returns the index of the closest point in `P` to the `Psimplex` points, according to the distance determined by the keyword argument `metric`.
Possible choices are:
 - `circumcenter`: (default) returns the point that minimize the circumradius
 - `dd`: like `circumcenter` but the circumradius is considered to be negative
    if the circumcenter is opposite to the new point with respect to `Psimplex`.

In [ ]:
function findClosestPoint(
		Psimplex::Lar.Points, P::Lar.Points;
		metric = "circumcenter"
	)::Union{Int64, Nothing}

	@assert metric ∈ ["circumcenter", "dd"] "findClosestPoint: available metrics are
		`circumcenter` and `dd`."

	simplexDim = size(Psimplex, 2)
	@assert simplexDim <= size(Psimplex, 1) "findClosestPoint: Cannot add
	another point to the simplex."

	@assert (m = size(P, 2)) != 0 "findClosestPoint: No Points in `P`."

	radlist = SharedArray{Float64}(m)
	@sync @distributed for col = 1 : m
		r, c = findRadius([Psimplex P[:,col]], true)
		sameSign = (
			r == Inf ||
			metric != "dd" ||
			isempty(AlphaStructures.oppositeHalfSpacePoints(
				[Psimplex P[:,col]], Psimplex, c
			))
		)
		radlist[col] = ((-1)^(1 + sameSign)) * r
	end

	radius, closestidx = findmin(radlist)

	if radius == Inf
		closestidx = nothing
	end

	return closestidx

end


#### Modified version 
For parallelize this function, we used the macros @spawn and @sync, as shown below:

In [ ]:
to "findClosestPoint" function findClosestPoint(
		Psimplex::Lar.Points, P::Lar.Points;
		metric = "circumcenter"
	)::Union{Int64, Nothing}

	@assert metric ∈ ["circumcenter", "dd"] "findClosestPoint: available metrics are
		`circumcenter` and `dd`."

	simplexDim = size(Psimplex, 2)
	@assert simplexDim <= size(Psimplex, 1) "findClosestPoint: Cannot add
	another point to the simplex."

	@assert (m = size(P, 2)) != 0 "findClosestPoint: No Points in `P`."

	radlist = SharedArray{Float64}(m)

	@sync for col = 1 : m
		rc = @spawn findRadius([Psimplex P[:,col]], true)
		r, c = fetch(rc)

		sameSign = (
			r == Inf ||
			metric != "dd" ||
			isempty(AlphaStructures.oppositeHalfSpacePoints(
				[Psimplex P[:,col]], Psimplex, c
			))
		)
		radlist[col] = ((-1)^(1 + sameSign)) * r
	end

	rc = @spawn findmin(radlist)
	radius, closestidx=fetch(rc)

	if radius == Inf
		closestidx = nothing
	end

	return closestidx

end

#### Original version of findRadius()
This function returns the value of the circumball radius of the given points.
If the function findCenter is not able to determine the circumcenter than the function returns `Inf`.
If the optional argument `center` is set to `true` than the function returns also the circumcenter cartesian coordinates.
_Obs._ Due to numerical approximation errors, the radius is choosen as the smallest distance between a point in `P` and the center.

In [ ]:
function findRadius(
		P::Lar.Points, center=false; digits=64
	)::Union{Float64, Tuple{Float64, Array{Float64,1}}}

 	c = AlphaStructures.findCenter(P)
	if any(isnan, c)
		r = Inf
	else
		r = round(
			findmin([Lar.norm(c - P[:, i]) for i = 1 : size(P, 2)])[1],
			digits = digits
		)
	end
	if center
		return r, c
	end
	return r
end

#### Original version of oppositeHalfSpacePoints()
This function returns the index list of the points `P` located in the halfspace defined by`face` points that do not contains the point `point`.
_Obs._ Dimension Dipendent, only works if dimension is three or less and
	the number of points in the face is the same than the dimension.

In [ ]:
function oppositeHalfSpacePoints(
		P::Lar.Points,
		face::Array{Float64,2},
		point::Array{Float64,1}
	)::Array{Int64,1}

	dim, n = size(P)
	noV = size(face, 2)
	@assert dim <= 3 "oppositeHalfSpacePoints: Not yet coded."
	@assert noV == dim "oppositeHalfSpacePoints:
		Cannot determine opposite to non hyperplanes."
	if dim == 1
		threshold = face[1]
		if point[1] < threshold
			opposite = [i for i = 1 : n if P[1, i] > threshold]
		else
			opposite = [i for i = 1 : n if P[1, i] < threshold]
		end
	elseif dim == 2
		if (Δx = face[1, 1] - face[1, 2]) != 0.0
			m = (face[2, 1] - face[2, 2]) / Δx
			q = face[2, 1] - m * face[1, 1]
			# false = under the line, true = over the line
			@assert point[2] ≠ m * point[1] + q "oppositeHalfSpacePoints,
				the point belongs to the face"
			side = sign(m * point[1] + q - point[2])
			opposite =
				[i for i = 1 : n if side * (m * P[1, i] + q - P[2, i]) < 0]
		else
			q = face[1, 1]
			side = sign(point[1] - q)
			opposite = [i for i = 1 : n if side * (P[1, i] - q) < 0]
		end


	elseif dim == 3
		axis = Lar.cross(
			face[:, 2] - face[:, 1],
			face[:, 3] - face[:, 1]
		)
		off = Lar.dot(axis, face[:, 1])
		position = Lar.dot(point, axis)
		if position < off
			opposite = [i for i = 1:size(P, 2) if Lar.dot(P[:,i], axis) > off]
		else
			opposite = [i for i = 1:size(P, 2) if Lar.dot(P[:,i], axis) < off]
		end
	end

	return [
		i for i in opposite
		if sum([P[:, i] == face[:, j] for j = 1 : noV]) == 0
	]

end

#### Modified version
For parallelize this function, we used @spawn

In [ ]:
function oppositeHalfSpacePoints(
		P::Lar.Points,
		face::Array{Float64,2},
		point::Array{Float64,1}
	)::Array{Int64,1}

	dim, n = size(P)
	noV = size(face, 2)
	@assert dim <= 3 "oppositeHalfSpacePoints: Not yet coded."
	@assert noV == dim "oppositeHalfSpacePoints:
		Cannot determine opposite to non hyperplanes."
	opposite=[]
	if dim == 1
		threshold = face[1]
		if point[1] < threshold
			#Per parallelizzare il metodo, abbiamo trasformato questo codice nel
			#codice che segue
			opposite = [i for i = 1 : n if P[1, i] > threshold]

			#@sync for i=1 : n
			#	if P[1,i] > threshold
			#		push!(opposite,i)
			#	end
			#end

		else
			#Per parallelizzare il metodo, abbiamo trasformato questo codice nel
			#codice che segue
			opposite = [i for i = 1 : n if P[1, i] < threshold]

			#@sync for i =1 : n
			#	if P[1,i]< threshold
			#		push!(opposite,i)
			#	end
			#end

		end
	elseif dim == 2
		if (Δx = face[1, 1] - face[1, 2]) != 0.0
			m = (face[2, 1] - face[2, 2]) / Δx
			q = face[2, 1] - m * face[1, 1]
			# false = under the line, true = over the line
			@assert point[2] ≠ m * point[1] + q "oppositeHalfSpacePoints,
				the point belongs to the face"
			side = sign(m * point[1] + q - point[2])
			#Per parallelizzare il metodo, abbiamo trasformato questo codice nel
			#codice che segue
			opposite =
				[i for i = 1 : n if side * (m * P[1, i] + q - P[2, i]) < 0]
			#@sync for i=1 : n
			#	if side * (m * P[1, i] + q - P[2, i]) < 0
			#		push!(opposite,i)
			#	end
			#end

		else
			q = face[1, 1]
			side = sign(point[1] - q)
			opposite = [i for i = 1 : n if side * (P[1, i] - q) < 0]

			 #@sync for i = 1 : n
			#	if side * (P[1, i] - q) < 0
			#		push!(opposite,i)
			#	end
			#end

		end

	elseif dim == 3
		axis = @spawn Lar.cross(
			face[:, 2] - face[:, 1],
			face[:, 3] - face[:, 1]
		)
		axis=fetch(axis)
		off = @spawn Lar.dot(axis, face[:, 1])
		off=fetch(off)
		position = @spawn Lar.dot(point, axis)
		position=fetch(position)
		#Per parallelizzare il metodo, abbiamo trasformato questo codice nel
		#codice che segue
		if position < off
			opposite = [i for i = 1:size(P, 2) if Lar.dot(P[:,i], axis) > off]
			#@sync for i=1 : size(P,2)
			#	if Lar.dot(P[:,i], axis) > off
			#		push!(opposite,i)
			#	end
			#end
		else
			opposite = [i for i = 1:size(P, 2) if Lar.dot(P[:,i], axis) < off]
			#@sync for i = 1:size(P, 2)
			#	if Lar.dot(P[:,i], axis) < off
			#		push!(opposite,i)
			#	end
			#end
		end
	end

	return [
		i for i in opposite
		if sum([P[:, i] == face[:, j] for j = 1 : noV]) == 0
	]
	"""
	faces=[]
	for i in opposite
		check= 0
		for j = 1 : noV
			if P[: , i] == face[:,j]
				check=check + 1
			end
		end
		if check == 0
			push!(faces,1)
		end
	end
	return faces
	"""
end